# Basics of transforms

This is  a walk through document to undertand transforms with notations.

1.Understanding representations in 2D, this involves rotation matrices and transsforms using homogenous representation in 2D

# Example 1 

Starting with a basic example
Assume in a world reference frames lies a cat at (1,0) 
and a mouse at (1,1)
Find  where the mouse lies in the cat's reference frame ?

In [ ]:
import numpy as np

# my solution : The mouse lies at (0.0, 1.0) in the cat's reference frame

# noting all the avilable transformations
# 1. T_cat_in_world 
# 2. T_mouse_in_world
# I need to find T_mouse_in_cat
# by chaining the transformations
# T_mouse_in_cat = T_cat_in_world * inv(T_mouse_in_world)

# 2D homogeneous transformation
def Rz_2d(theta):
    return np.array([[np.cos(theta), -np.sin(theta)],
                     [np.sin(theta), np.cos(theta)]])
    
def T_x_in_y_2d(theta, T):
    # Check if T has exactly 2 elements (works for row or column vectors)
    arr = np.asarray(T)
    if arr.size == 2:
        _T_x_in_y = np.eye(3)
        _T_x_in_y[0:2, 0:2] = Rz_2d(theta)
        _T_x_in_y[0:2, 2] = T
        return _T_x_in_y
    else:
        raise ValueError("T must be a 2D vector (x, y) for 2D transformations.")

# 1. T_cat_in_world 
T_cat_in_world = T_x_in_y_2d(0, [1, 0])


# 2. T_mouse_in_world
T_mouse_in_world = T_x_in_y_2d(0, [1, 1])


# 3. T_mouse_in_cat
T_mouse_in_cat = np.matmul(np.linalg.inv(T_cat_in_world),T_mouse_in_world) 

print(f"The mouse lies at ({T_mouse_in_cat[0, 2]}, {T_mouse_in_cat[1, 2]}) in the cat's reference frame")


The mouse lies at (0.0, 1.0) in the cat's reference frame


# Example 2

In the previous example it was given the mouse was looking in the same direction the cat was , and the world reference was aligned to them , here its a bit different. Now the cat is 90 degs rotated wrt to the world but is still looking at the mouse which is running awy in the same direction. Along with the position of the mouse in the cats reference, now find the world reference frame in the mouse frmae of reference


In [ ]:
# in the previous ti,e the theta was 0 but now that changes hence

# 1. T_cat_in_world (wTc)
T_cat_in_world = T_x_in_y_2d(np.deg2rad(90), [1, 0])

# 2. T_mouse_in_world (wTm)
T_mouse_in_world = T_x_in_y_2d(np.deg2rad(90), [1, 1])

# 3. T_mouse_in_cat ( cTm = cTw * wTm) ==> very important to digest
T_mouse_in_cat = np.matmul(np.linalg.inv( T_cat_in_world),T_mouse_in_world)

# 4. T_world_in_mouse
T_world_in_mouse = np.linalg.inv(T_mouse_in_world)

print(f"The mouse lies at ({T_mouse_in_cat[0, 2]}, {T_mouse_in_cat[1, 2]}) in the cat's reference frame")
print(f"The world ref lies at ({T_world_in_mouse[0, 2]}, {T_world_in_mouse[1, 2] }) in the mouse's reference frame")


# expected answer is (1, 0) in the cat's reference frame

# wTc wTm
# Summary
# Premultiplying by the inverse of the cat-in-world transform 
# is necessary to express the mouse's pose in the cat's frame,
# because you are "moving" from the world frame into the cat's frame,
# then applying the mouse's position in the world. 
# This is standard in coordinate frame chaining.

The mouse lies at (0.9999999999999999, 1.1102230246251565e-16) in the cat's reference frame
The world ref lies at (-1.0, 0.9999999999999999) in the mouse's reference frame


# Example 3

Now lets try a 3D problem , now the cat is at (1,0,0) in the world frame and the cat turned 90 deg is looking up 45 deg to see a mouse  on a shelf at (1, 1, 1) w.R.t the world. Note the mouse frame is rotated 180 deg to the world.

In [ ]:
import numpy as np
# 3D homogeneous transformation
def Rx(roll):
    return np.array([[1,        0,              0  ],
                     [0, np.cos(roll),-np.sin(roll)],
                     [0, np.sin(roll), np.cos(roll)]])

def Ry(pitch):
    return np.array([[np.cos(pitch),    0,      np.sin(pitch) ],
                     [0,               1.0,           0.      ],
                     [-np.sin(pitch),   0.,      np.cos(pitch)]])
    
def Rz(yaw):
    return np.array([[np.cos(yaw),-np.sin(yaw), 0.],
                     [np.sin(yaw), np.cos(yaw), 0.],
                     [0.,          0.,          1.]])
    
def T_x_in_y(angles, T):
    """
    Create a homogeneous transformation matrix from frame x to frame y.
    angles: tuple of (yaw, pitch, roll) in radians
    T: translation vector from frame x to frame y (should be a 3D vector)
    """
    # Check if T has exactly 2 elements (works for row or column vectors)
    arr = np.asarray(T)
    if arr.size == 3:
        _T_x_in_y = np.eye(4)
        _T_x_in_y[0:3, 0:3] = np.matmul(np.matmul(Rz(angles[0]), Ry(angles[1])), Rx(angles[2]))
        _T_x_in_y[0:3, 3] = T
        return _T_x_in_y
    else:
        raise ValueError("T must be a 3D vector (x, y, z) for 3D transformations.")
    
# 1. T_cat_in_world (wTc)
T_cat_in_world = T_x_in_y([np.deg2rad(90), np.deg2rad(-45), 0], [1, 0, 0])

# 2. T_mouse_in_world (wTm)
T_mouse_in_world = T_x_in_y([np.deg2rad(180), 0, 0], [1, 1, 1])

# 3. T_mouse_in_cat (cTm = cTw * wTm)
T_mouse_in_cat = np.matmul(np.linalg.inv(T_cat_in_world), T_mouse_in_world)
print(f"The mouse lies at ({T_mouse_in_cat[0, 3]}, {T_mouse_in_cat[1, 3]}, {T_mouse_in_cat[2, 3]}) in the cat's reference frame")

The mouse lies at (1.414213562373095, 1.1102230246251565e-16, 1.543201052742903e-16) in the cat's reference frame


# Example 4

Moving on to some real world examples, there is a tractor  with a GNSS antenna and we want to track the tractor reference frame which is between the tractors rear wheels. Given the antenna is positioned at (1,1,1) in the tractor reference and the GNSS reports a position of (0,0,0) with a heading of 90 deg.  Where is the tractor reference in the world? 

In [ ]:
# 1. T_gnss_in_world (wTg)
T_gnss_in_world = T_x_in_y([np.deg2rad(90), 0, 0], [0., 0., 0.])

# 2. T_gnss_in_tractor (tTg)
T_gnss_in_tractor = T_x_in_y([0, 0, 0], [1., 1., 1.]) 

# 3. T_tractor_in_world (wTt = wTg * gTt )
T_tractor_in_world = np.matmul(T_gnss_in_world, np.linalg.inv(T_gnss_in_tractor))

print(f"The tractor ref lies at ({T_tractor_in_world[0, 3]}, {T_tractor_in_world[1, 3]}, {T_tractor_in_world[2, 3]}) in the world reference frame")



The tractor lies at (0.9999999999999999, -1.0, -1.0) in the world reference frame
